# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')));

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

#### 1. Find the 10 countries with most projects

In [8]:
# load json as string
json_load = json.load((open('data/world_bank_projects.json')))
json_load;

In [9]:
# load as Pandas dataframe
json_df = pd.read_json('data/world_bank_projects.json')
json_df;

In [10]:
# count values of countries and only the top 10
json_df.countryname.value_counts().head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

#### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [11]:
# to get an idea of how 'mjtheme_namecode' is listed out
json_df.mjtheme_namecode;

In [12]:
# normalize using json_load
themes = json_normalize(json_load, 'mjtheme_namecode')
themes;

In [13]:
# able to see how the name column is listed out and we notice there are some blanks
themes.name;

In [14]:
# The blanks are counted as 1 item
themes.name.value_counts()

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: name, dtype: int64

In [15]:
# shows the empty values are not represented as null
themes.name.notnull();

In [16]:
# Allows us to only show rows that aren't blank for the name column
themes[themes.name != ''].name;

In [17]:
# count values of major projects and only the top 10
themes[themes.name != ''].name.value_counts().head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Name: name, dtype: int64

#### 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [18]:
themes2 = themes

In [19]:
# List unique values of themes2
themes2.code.unique()

array(['8', '11', '1', '6', '5', '2', '7', '4', '10', '9', '3'], dtype=object)

In [20]:
# Allows us to see the corresponding name for code - adjust '11' to desired number
themes2.loc[themes2['code'] == '11'].head(2)

,code,name
1,11,
6,11,Environment and natural resources management


In [21]:
# Assign each unique value in 'code' its corresponding name
themes2.loc[themes2.code == '1', 'name'] = 'Economic management'
themes2.loc[themes2.code == '2', 'name'] = 'Public sector governance'
themes2.loc[themes2.code == '3', 'name'] = 'Rule of law'
themes2.loc[themes2.code == '4', 'name'] = 'Financial and private sector development'
themes2.loc[themes2.code == '5', 'name'] = 'Trade and integration'
themes2.loc[themes2.code == '6', 'name'] = 'Social protection and risk management'
themes2.loc[themes2.code == '7', 'name'] = 'Social dev/gender/inclusion'
themes2.loc[themes2.code == '8', 'name'] = 'Human development'
themes2.loc[themes2.code == '9', 'name'] = 'Urban development'
themes2.loc[themes2.code == '10', 'name'] = 'Rural development'
themes2.loc[themes2.code == '11', 'name'] = 'Environment and natural resources management'

In [22]:
# Can see changes at index 1
themes2;

In [23]:
# Can see there is no blank value
themes2.name.value_counts()

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
Name: name, dtype: int64

In [24]:
# list the true top 10 themes
themes2.name.value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64